# **IMPORT THE MODEL**

## Kalau Pake Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download Labelled Dataset from Roboflow

## 0. Installing the Dependencies

In [7]:
#!pip install ultralytics opencv-python pillow

In [1]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

## Optional : Check Whether CUDA is Available or Not (or else will using the CPU as computation source)

In [5]:
!nvidia-smi

Fri Jul  5 20:52:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   53C    P3             23W /   55W |    1752MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch

# Check if CUDA is available and set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


## 1. Data Preprocessing

### Convert TIF images to PNG or JPG

In [ ]:
from PIL import Image
import os

def convert_tiff_to_jpg(tiff_path, output_path):
    with Image.open(tiff_path) as img:
        img.convert('RGB').save(output_path, format='JPEG')  # Bisa diganti ke PNG di format = 'PNG'

def batch_convert_tiff_to_jpg(input_folder_path, output_folder_path):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    for filename in os.listdir(input_folder_path):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            tiff_path = os.path.join(input_folder_path, filename)
            output_filename = filename.replace('.tif', '.jpg').replace('.tiff', '.jpg')  # Bisa diganti ke PNG (ubah '.jpg' -> '.png')
            output_path = os.path.join(output_folder_path, output_filename)
            convert_tiff_to_jpg(tiff_path, output_path)

# Example usage:
input_folder_path = r'D:\perjuanganHH\Kuliah\code\Road-Damage-Detection\640x640\kerusakan' #Directory Folder TIF
output_folder_path = r'D:\perjuanganHH\Kuliah\code\Road-Damage-Detection\640x640\converted_images' #Directory Folder Output
batch_convert_tiff_to_jpg(input_folder_path, output_folder_path)

## 2. Model Training

In [ ]:
dataset_location = "D:\perjuanganHH\Kuliah\code\Road-Damage-Detection\Pothole and Crack Detection.v4i.yolov8\data.yaml" # Ganti Directory Sesuai Dengan Directory Asli Dataset

In [ ]:
from ultralytics import YOLO
import os

# Load a model
model = YOLO('yolov8s-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8s-seg.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8s-seg.yaml').load('yolov8s-seg.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data=dataset_location, epochs=110, imgsz=640, device = device)

## 3. Model Validation

In [ ]:
# Load a model
model = YOLO('D:\perjuanganHH\Kuliah\code\\runs\segment\\train25\weights\\best.pt')  # Ganti dengan Directory Model Hasil Training (Biasanya bakalan ditunjukin dimana nyimpennya pas selesai training)

# Validate the model
metrics = model.val(data = dataset_location)  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

## 4. Testing to Real Images

In [ ]:
import cv2
import os
from ultralytics import YOLO

# Load the model
model = YOLO(r'D:\perjuanganHH\Kuliah\code\runs\segment\train25\weights\best.pt')  # Ganti dengan Directory Model Hasil Training (Biasanya bakalan ditunjukin dimana nyimpennya pas selesai training)

# Run batched inference on a list of images
results = model(r"D:\perjuanganHH\Kuliah\code\Road-Damage-Detection\640x640\640x640\PNG\*.png")  # Ganti dengan Directory Folder Gambar yang Ingin Dideteksi, format bisa jpg/png (tinggal diubah di "*.png atau *.jpg")

# Specify the directory to save the results
save_dir = r"D:\perjuanganHH\Kuliah\code\Road-Damage-Detection\Model After Train\Yolov8-Result-New-2" #Ganti dengan Directory Folder Hasil Deteksi
os.makedirs(save_dir, exist_ok=True)

# Process results generator
for idx, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

    # Get the original image
    img = result.orig_img

    # Draw bounding boxes and masks on the image
    annotated_img = result.plot()  # This method is available in YOLOv5

    # Save the result image
    save_path = os.path.join(save_dir, f'result_{idx}.jpg')
    cv2.imwrite(save_path, annotated_img)

print(f"Images saved in directory: {os.path.abspath(save_dir)}")

## 5. Convert the Result Back to TIF

In [ ]:
import os
from PIL import Image
from osgeo import gdal, osr
import numpy as np
import glob

def convert_png_to_tiff(png_path, output_tiff_path, geotiff_template):
    # Open the PNG image
    img = Image.open(png_path)

    # Get image size (width, height)
    width, height = img.size

    # Convert PIL image to numpy array
    img_array = np.array(img)

    # Get the geotransform and projection from the template GeoTIFF
    dataset = gdal.Open(geotiff_template, gdal.GA_ReadOnly)
    if dataset is None:
        raise ValueError(f"Failed to open template GeoTIFF file: {geotiff_template}")

    geotransform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()
    srs = osr.SpatialReference(wkt=projection)

    # Create a new GeoTIFF dataset
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_tiff_path, width, height, 3, gdal.GDT_Byte)
    new_dataset.SetGeoTransform(geotransform)
    new_dataset.SetProjection(srs.ExportToWkt())

    # Write the image data to the new GeoTIFF dataset
    for i in range(3):  # For RGB bands
        band = new_dataset.GetRasterBand(i + 1)  # Bands are 1-indexed
        band.WriteArray(img_array[:,:,i])  # Write each channel separately

    # Close datasets
    dataset = None
    new_dataset = None

def batch_convert_png_to_tiff(png_paths, geotiff_templates, output_dir):
    # Print sorted PNG file paths
    print("Sorted PNG file paths:")
    for png_path in png_paths:
        print(png_path)

    # Print sorted GeoTIFF template paths
    print("\nSorted GeoTIFF template paths:")
    for geotiff_template in geotiff_templates:
        print(geotiff_template)

    # Perform batch conversion
    for png_path, geotiff_template in zip(png_paths, geotiff_templates):
        base_name = os.path.basename(geotiff_template)
        output_tiff_path = os.path.join(output_dir, base_name)
        convert_png_to_tiff(png_path, output_tiff_path, geotiff_template)
        print(f"Converted: {png_path} -> {output_tiff_path}")

# Example usage:
png_paths = sorted(glob.glob('/content/drive/MyDrive/Road Damage Detection/Result-Yolov8-JPG/result_*.jpg'), key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1])) #Diganti dengan Directory Folder Hasil Deteksi
geotiff_templates = sorted(glob.glob('/content/drive/MyDrive/Road Damage Detection/Dataset TIF/Data_*.tif')) # Diganti dengan Directory Folder TIF Asli

output_dir = '/content/drive/MyDrive/Road Damage Detection/Result-Yolov8-TIF'

if len(png_paths) != len(geotiff_templates):
    raise ValueError("The number of PNG files does not match the number of GeoTIFF templates.")

os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it does not exist

batch_convert_png_to_tiff(png_paths, geotiff_templates, output_dir)